In [7]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import keras
import random
import cv2
import math
import seaborn as sns
import matplotlib.pyplot as plt
import shutil

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator



In [2]:
train_path = '/content/drive/MyDrive/Paper/Structure_data/train'
val_path = '/content/drive/MyDrive/Paper/Structure_data/val'
test_path = '/content/drive/MyDrive/Paper/Structure_data/test'

In [3]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=40, width_shift_range=0.2,
    height_shift_range=0.2).flow_from_directory(train_path, target_size=(224, 224), batch_size=32, class_mode='sparse')

val_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=40, width_shift_range=0.2,
    height_shift_range=0.2).flow_from_directory(val_path, target_size=(224, 224), batch_size=32, class_mode='sparse')


Found 2074 images belonging to 3 classes.
Found 450 images belonging to 3 classes.


In [5]:
image_labels = sorted(os.listdir(train_path))
num_classes = len(image_labels)
print(num_classes)

3


In [ ]:
base_model = tf.keras.applications.MobileNetV3Large(input_shape=(224, 224, 3), weights='imagenet',
                                                     include_top=False)
base_model.summary()


In [ ]:

for layer in base_model.layers:
    layer.trainable = False

last_output = base_model.layers[-1].output
x = tf.keras.layers.Dropout(0.45)(last_output)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dense(224, activation='relu')(x)
x = tf.keras.layers.Dropout(0.45)(x)
x = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

print('compiling model')
model = tf.keras.Model(base_model.input, x)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00125),
              loss='sparse_categorical_crossentropy', metrics=['accuracy'])
epochs = 20
#lrCallback = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 1e-3 * 10 ** (epoch / 30))
#stepDecay = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 0.1 * 0.1**math.floor(epoch / 6))
#callbacks = tf.keras.callbacks.EarlyStopping(patience=2)
check_point = tf.keras.callbacks.ModelCheckpoint(filepath='MobileNet3.h5', save_best_only=True)
history = model.fit(train_generator, validation_data=val_generator, epochs=epochs, callbacks=[check_point])

print('traning done')

compiling model
Epoch 1/20


In [ ]:
test_generator = tf.keras.preprocessing.image.ImageDataGenerator().flow_from_directory(test_path, target_size=(224, 224), batch_size=1, shuffle=False, class_mode='sparse')

In [ ]:
# Load the saved model
loaded_model = tf.keras.models.load_model('/content/drive/MyDrive/CNN/Model/MobileNet3.h5')

# Predict the images in the test set
predictions = loaded_model.predict(test_generator)